In [8]:
import pandas as pd
import numpy as np
data = pd.read_excel('text.xlsx')
data

,product,csss,target,polarity
0,a,cs,bc,1
1,a,ss,bc,1
2,a,cs,bc,1
3,a,cs,bd,1
4,b,cs,cd,1
5,b,cs,cd,1
6,b,cs,cd,1
7,b,cs,ce,1
8,c,cs,ad,1
9,c,cs,ae,1


In [6]:
product_list = ['a','b','c','d','e']
def Make_matrix(df, product_list = product_list):
    matrix = np.zeros((len(product_list), len(product_list)), dtype=np.float32)
    for i, p in enumerate(product_list):
        df_re = df[df['product'] == p]
        df_cs = df_re[df_re['csss'] == 'cs']
        df_ss = df_re[df_re['csss'] == 'ss']
        
        for j, t in enumerate(product_list):
            if i == j:
                # ss에 의한 웨이트 계산
                ss_data = pd.DataFrame(df_ss['polarity'].value_counts())
                if len(ss_data.index) == 2:
                    matrix[i,i] = float(ss_data.loc[1] / ss_data.loc[-1])
                elif 1 not in ss_data.index:
                    matrix[i,i] = 0
                else:
                    matrix[i,i] = float(ss_data.loc[1])

            elif i != j:
                # cs에 의한 웨이트 계산
                cs_data = pd.DataFrame(df_cs[df_cs['target'].str.contains(t)]['polarity'].value_counts())
                if len(cs_data.index) == 2:
                    matrix[i,j] = float(cs_data.loc[1] / cs_data.loc[-1])
                elif 1 not in cs_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = float(cs_data.loc[1])

    return matrix

In [57]:
a = Make_matrix(data)
a

array([[3., 3., 4., 3., 0.],
       [0., 2., 4., 3., 1.],
       [4., 0., 0., 2., 2.],
       [2., 4., 2., 0., 0.],
       [0., 1., 1., 2., 2.]], dtype=float32)

In [60]:
dff = pd.DataFrame(a, index = product_list, columns = product_list)
dff.loc['sum'] = dff.sum()

In [61]:
dff

,a,b,c,d,e
a,3.0,3.0,4.0,3.0,0.0
b,0.0,2.0,4.0,3.0,1.0
c,4.0,0.0,0.0,2.0,2.0
d,2.0,4.0,2.0,0.0,0.0
e,0.0,1.0,1.0,2.0,2.0
sum,9.0,10.0,11.0,10.0,5.0


In [54]:
score = np.ones(len(product_list))
score

array([1., 1., 1., 1., 1.])

In [80]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    score = np.ones(len(product_list))

    for iter in range(0,MAX_ITERATIONS):
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            summation = 0
            for j, t in enumerate(product_list):
                if df.loc['sum', t] != 0:
                    summation += df.loc[p, t] / df.loc['sum', t] * score[j]

            score[i] = (1-d) + d*summation

        if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
            print('요기서 끝 :', iter)
            break
    
    dff = pd.DataFrame(score, index = product_list)
    sum_num = float(dff.sum())
    dff = dff.apply(lambda x: x/sum_num)
    
    return dff

In [81]:
a = Make_Rank(dff)
a

요기서 끝 : 27


,0
a,0.267681
b,0.199098
c,0.210218
d,0.180742
e,0.142261
